## Importing Important Libraries

### Steps To Be Followed

In [13]:
import sagemaker
import boto3 # we can read the s3 bucket from our local environment if it is public using boto3 Algorithm
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.session import s3_input, Session

In [20]:
bucket_name = 'bankkapplication' #crreating a bucket
my_region= boto3.session.Session().region_name #checking my region name because based on region name i may be accessing different buckets
print(my_region)

us-east-1


In [21]:
s3 = boto3.resource('s3') # acessing my s3 bucket
try:
    if my_region == 'us-east-1':
        s3.create_bucket(Bucket=bucket_name) # creating bucket using code
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error:',e)

S3 bucket created successfully


In [22]:
# set an output path where the trained model will be saved
prefix = 'xgboost-as-a-built-in-algo'
output_path = 's3://{}/{}/output'.format(bucket_name, prefix)
print(output_path)

s3://bankkapplication/xgboost-as-a-built-in-algo/output


### Downloading the dataset and storing in S3

In [24]:
import pandas as pd
import urllib
try:
    # downloading the dataset from this location and renaming this.
    urllib.request.urlretrieve ("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
    print('Success: downloaded bank_clean.csv.')
except Exception as e:
    print('Data load error: ',e)

try:
    3reading the dataset
    model_data = pd.read_csv('./bank_clean.csv',index_col=0)
    print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: downloaded bank_clean.csv.
Success: Data loaded into dataframe.


In [25]:
# Splitting tarining and testing data
import numpy as np
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
print(train_data.shape, test_data.shape)

(28831, 61) (12357, 61)


In [27]:
### Saving Train And Test Into Buckets
## We start with Train Data
import os
import sagemaker
from sagemaker.inputs import TrainingInput
# When you are working with sagemaker, the dependent should be the 1st column
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], 
                                                axis=1)], 
                                                axis=1).to_csv('train.csv', index=False, header=False)
#uploading train.csv to s3 bucket
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
# path of the training data
s3_input_train = TrainingInput(
    s3_data='s3://{}/{}/train/train.csv'.format(bucket_name, prefix),
    content_type='text/csv'
)

[01/07/25 21:23:37] INFO     Found credentials from IAM Role:                                   ]8;id=54772;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=163719;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

In [30]:
# Test data into buckets
# Test Data Into Buckets
pd.concat([test_data['y_yes'], test_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('test.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')
s3_input_test = TrainingInput(
    s3_data='s3://{}/{}/test/test.csv'.format(bucket_name, prefix),
    content_type='text/csv'
)

[01/07/25 21:37:31] INFO     Found credentials from IAM Role:                                   ]8;id=831824;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=149068;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

### Building and Training Models Xgboost- Inbuilt Algorithm

In [31]:
# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
# specify the repo_version depending on your preference.
from sagemaker.image_uris import retrieve

container = retrieve(
    framework='xgboost',
    region=boto3.Session().region_name,
    version='1.0-1'
)


[01/07/25 21:42:10] INFO     Defaulting to only available Python version: py3                     ]8;id=89757;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=855566;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py#608\608]8;;\

                    INFO     Defaulting to only supported image scope: cpu.                       ]8;id=807458;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=252994;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py#532\532]8;;\

In [32]:
# Hyper parameter tunning
# initialize hyperparameters
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"binary:logistic",
        "num_round":50
        }

In [37]:
from sagemaker.estimator import Estimator

estimator = Estimator(
    image_uri=container,  # Updated parameter name
    hyperparameters=hyperparameters,
    role=sagemaker.get_execution_role(),
    train_instance_count=1,
    train_instance_type='ml.m5.2xlarge',
    train_volume_size=5,  # 5 GB
    output_path=output_path,
    train_use_spot_instances=True,
    train_max_run=300,
    train_max_wait=600
)


[01/07/25 21:59:34] WARNING  train_instance_count has been renamed in sagemaker>=2.              ]8;id=86062;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/deprecations.py\deprecations.py]8;;\:]8;id=784920;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/deprecations.py#34\34]8;;\
                             See: https://sagemaker.readthedocs.io/en/stable/v2.html for                           
                             details.                                                                              

                    WARNING  train_instance_type has been renamed in sagemaker>=2.               ]8;id=962202;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/deprecations.py\deprecations.py]8;;\:]8;id=701991;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/deprecations.py#34\34]8;;\
                             See: https://sagemaker.readthedocs.io/en/stable/v2.html for                           
                             details.                                                                              

                    WARNING  train_max_run has been renamed in sagemaker>=2.                     ]8;id=987693;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/deprecations.py\deprecations.py]8;;\:]8;id=216359;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/deprecations.py#34\34]8;;\
                             See: https://sagemaker.readthedocs.io/en/stable/v2.html for                           
                             details.                                                                              

                    WARNING  train_use_spot_instances has been renamed in sagemaker>=2.          ]8;id=668646;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/deprecations.py\deprecations.py]8;;\:]8;id=107366;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/deprecations.py#34\34]8;;\
                             See: https://sagemaker.readthedocs.io/en/stable/v2.html for                           
                             details.                                                                              

                    WARNING  train_max_wait has been renamed in sagemaker>=2.                    ]8;id=643418;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/deprecations.py\deprecations.py]8;;\:]8;id=407498;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/deprecations.py#34\34]8;;\
                             See: https://sagemaker.readthedocs.io/en/stable/v2.html for                           
                             details.                                                                              

                    WARNING  train_volume_size has been renamed in sagemaker>=2.                 ]8;id=975623;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/deprecations.py\deprecations.py]8;;\:]8;id=786341;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/deprecations.py#34\34]8;;\
                             See: https://sagemaker.readthedocs.io/en/stable/v2.html for                           
                             details.                                                                              

In [38]:
estimator.fit({'train': s3_input_train,'validation': s3_input_test})

[01/07/25 21:59:59] INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=820316;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=854090;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py#90\90]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

                    INFO     Creating training-job with name:                                       ]8;id=439064;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=667313;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#1042\1042]8;;\
                             sagemaker-xgboost-2025-01-07-21-59-59-328                                             

2025-01-07 22:00:00 Starting - Starting the training job...
2025-01-07 22:00:15 Starting - Preparing the instances for training...
2025-01-07 22:00:54 Downloading - Downloading the training image...
2025-01-07 22:01:30 Training - Training image download completed. Training in progress....
2025-01-07 22:02:03 Uploading - Uploading generated training model
2025-01-07 22:02:03 Completed - Training job completed
[2025-01-07 22:01:47.338 ip-10-2-203-74.ec2.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:D

### Deploy Machine Learning Model As Endpoints

In [39]:
xgb_predictor = estimator.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

[01/07/25 22:04:11] INFO     Creating model with name: sagemaker-xgboost-2025-01-07-22-04-10-958    ]8;id=940614;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=694513;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4094\4094]8;;\

                    INFO     Creating endpoint-config with name                                     ]8;id=714948;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=161838;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#5889\5889]8;;\
                             sagemaker-xgboost-2025-01-07-22-04-10-958                                             

                    INFO     Creating endpoint with name sagemaker-xgboost-2025-01-07-22-04-10-958  ]8;id=882181;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=411175;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4711\4711]8;;\

------!

### Prediction of the Test Data

In [41]:
from sagemaker.serializers import CSVSerializer
import numpy as np

# Prepare the test data
test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values  # Load the data into an array

# Configure the XGBoost predictor
xgb_predictor.content_type = 'text/csv'  # Set the content type for inference
xgb_predictor.serializer = CSVSerializer()  # Use CSVSerializer for the serializer type

# Perform predictions
predictions = xgb_predictor.predict(test_data_array).decode('utf-8')  # Predict!
predictions_array = np.fromstring(predictions[1:], sep=',')  # Convert predictions into an array

# Print predictions shape
print(predictions_array.shape)


(12357,)


In [42]:
predictions_array

array([0.05214286, 0.05660191, 0.05096195, ..., 0.03436061, 0.02942475,
       0.03715819])

In [43]:
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))
     


Overall Classification Rate: 89.7%

Predicted      No Purchase    Purchase
Observed
No Purchase    91% (10785)    34% (151)
Purchase        9% (1124)     66% (297) 

